Sequence-Prediction report: Ben Dawson

The purpose of this notebook is to report on my findings of using machine learning to predict customer movements to provide insights to aquarium/zoo owners. I have implemented and tested the following strategies:
1) Markov Chains
2) k-Nearest Neighbours
3) Recurrent Neural Network

Markov Chains
Markov Chains use a transition matrix to determine the next state, given the current state. They do this by using past data to determine where the majority of past customers' next state was, and therefore assuming that is where the customer will go next. My method functions like this, however the matrix stores the amount of times customers partook in the transition as opposed to the probability of going to the next location. This provides more insight to zoo owners and reduces computation time, while still operating the same way. The matrix is stored in a json file for persistence.

In [ ]:
from area import Area
from typing import List, Tuple
import json 

#Makes a prediction where the user will go next given the current location
def nextArea(current: str) -> str:

    matrix = loadMarkovChain("matrix.json")
    
    if current in matrix:
        counts = matrix[current]
        nextState = max(counts, key=counts.get)
        return nextState
    else:
        return None

#Updates the counts for the matrixes to update the probabilities
def updateMatrix(matrix: dict, data: List[Tuple[str, str]], filename: str):
    
    for transition in data:
        currentState, nextState = transition
        if currentState in matrix and nextState in matrix[currentState]:
            matrix[currentState][nextState] += 1
    saveMatrix(filename, matrix)
 
#Creates an empty transition matrix, setting all counts to 0
def createMarkovChain(attractionAreas: List[Area], filename: str):

    areas = areasToStrings(attractionAreas)
    
    transitionMatrix = {}

    for area in areas:
        transitionMatrix[area] = {otherArea: 0.0 for otherArea in areas if otherArea != Area}

    with open (filename, 'w') as file:
        json.dump(transitionMatrix, file, indent=4)

#Loads the transition matrix from a json file so it can be used in the program
def loadMarkovChain(filename: str) -> dict:
    with open(filename, 'r') as file:
        transitionMatrix = json.load(file)
        return transitionMatrix
    
#Converts a list of Area objects into a list of strings, where the strings are the object names
def areasToStrings(path: List[Area]) -> List[str]:
    areas: List[str] = []
    for area in path:
        areas.append(area.get_name())
    return areas

#Generates transitions in the form (area, area) to be added to the transition matrix to update probabilities
def generateTransitions(path: List[Area]) -> List[Tuple[str, str]]:

    areas = areasToStrings(path)

    transitions = []

    for i in range(len(areas) - 1):
        transitions.append((areas[i], areas[i+1]))
    
    return transitions

#Function to save the transition matrix
def saveMatrix(filename: str, matrix: dict):
    with open (filename, 'w') as file:
        json.dump(matrix, file, indent=4)

Pros:
- Storage(size of json file) and computation time does not rise with the quantity of user data and instead with the number of attractions (this has a minimal effect)
- Logical

Cons:
- Inefficient if the entrance and exit of the park is the same. For example, if entrance and exit is the same, while customers may go to different places from the entrance at the start of the trip, they will likely all go past the same attractions to leave the park. This may skew the transition matrix, and suggest the most likely course is for someone to immediately leave as previous movements not accounted for
- Does not account for previous movements


k-Nearest Neighbours is an approach that also uses past customer paths to guess where the customer will go next. The below code stores all user paths in a csv file and loads them and gives each one a score between 0 and 1 with 0 being no similarity, 1 being the exact same to the inputted path. the top 10 scores are used to work out which area the user will go to next. The scoring system has a recentcy bias, therefore the earlier paths have less weighting to where the user will go next

In [ ]:
import pandas as pd
from pandas import DataFrame
from area import Area
from typing import List, Tuple
import csv
from collections import Counter

def nearestNeighbour(filename: str, path: List[Area]) -> str:
    
    df = loadCSV(filename)

    pathStr = convertPath(path)

    addPathToCSV(filename, pathStr)
    
    if len(pathStr) >= len(df.columns):
        return None
    
    neighbours: List[Tuple[float, str]] = []

    for index, row in df.iterrows():
        if len(pathStr) < len(row):
            rowList = [str(item) for item in row]
            neighbours.append(comparePaths(pathStr, rowList))

    sortedNeighbours = sorted(neighbours, key=lambda x: x[0], reverse=True)

    topValues = [x[0] for x in sortedNeighbours[:10]]

    areaCounter = Counter()
    for value, string in sortedNeighbours:
        if value in topValues:
            areaCounter[string] += 1
    
    mostCommonArea = areaCounter.most_common(1)
    if mostCommonArea:
        mostCommonArea = mostCommonArea[0][0]
    else:
        mostCommonArea = None
    return mostCommonArea

def loadCSV(filepath: str) -> DataFrame:

    #Calculates the max amount of columns in the file i.e: the longest path
    maxColumns = 0
    with open (filepath, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            maxColumns = max(maxColumns, len(row))

    #Apply padding to rows so effectively each row has the same length
    with open (filepath, 'r') as file:
        reader = csv.reader(file)
        rows = []
        for row in reader:
            paddedRow = row + [''] * (maxColumns - len(row))
            rows.append(paddedRow)
    
    return pd.DataFrame(rows)


#Converts the path in the form of a list of area objects into a list of strings
def convertPath(path: List[Area]) -> List[str]:
    areas = []
    for area in path:
        areas.append(area.get_name())
    return areas

#Calculates the similarity score and returns a tuple containing this score, and the next predicted area
def comparePaths(path: List[str], row:List[str] ) -> Tuple[float, str]:
    maxScore = (2 ** len(path)) - 1
    score = 0

    for i in range(len(path)):
        if path[i] == row[i]:
            score = score + (2 ** i)

#Ensures that the score is a value between 0 and 1
    normalisedScore = round(score / maxScore, 3)    
    result = (normalisedScore, row[len(path)])
    return result


def addPathToCSV(filename: str, path: List[str]):
    df = pd.DataFrame([path])
    df.to_csv(filename, mode='a', header=False, index=False)

 

Pros:
- Recency bias to make it more accurate as the longer the path is, the more unique the similarity is so more likely to go to the same place
Cons:
- High time complexity of compuation; scales with the quantity of data
- Memory intensive as all data must be stored permanently

Recurrent Neural Networks are specialised at processing and predicting sequential data. One of the ways they do this is by parameter sharing, which enables the model to work with paths with varying lengths, which would be common when tracking customer movement.

In [ ]:
import numpy as np 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from area import Area
from typing import List

def loadRNN(sequences: List[List[str]], rnnUnits: int = 128, embeddingDim: int = 50):

    areas = [location for seq in sequences for location in seq] #List of unique areas

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(areas) #Converts each location to an integer
    locationIndex = tokenizer.word_index #Index for dictionary 
    numLocations = len(locationIndex)

    paths = tokenizer.texts_to_sequences(sequences)
    x = [seq[:-1] for seq in paths] #Input sequences(all but the last area on the path)
    y = [seq[-1] for seq in paths] #Next location to predict

    sequenceLength = max(len(seq) for seq in x)
    x_padded = pad_sequences(x, maxlen=sequenceLength, padding='pre') #Pads each sequence to be the same length

    x_padded = np.array(x_padded)
    y = np.array(x)

    model = Sequential([
    Embedding(input_dim=numLocations + 1, output_dim=embeddingDim, input_length=sequenceLength),
    LSTM(rnnUnits, return_sequences=False),
    Dense(numLocations + 1, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    model.fit(x_padded, y, epochs=10, batch_size=32, validation_split=0.2)

    return model

def summariseModel(model):
    model.summary()

def evaluateModel(model, x, y):
    loss, accuracy = model.evaluate(x, y)
    print(f'Test Loss: {loss}')
    print(f'Test Accuracy: {accuracy}')


def predictNextArea(path: List[str], model, size: int) -> str:
    pathEncoded = tokenizer.texts_to_sequences([path])
    pathPadded = pad_sequences(pathEncoded, maxlen=path, padding='pre')
    predictedArea = model.predict(pathPadded)
    predictedAreaID = np.argmax(predictedArea, axis=1)[0]
    predictedAreaName = tokenizer.index_word[predictedAreaID]

    return predictedAreaName


Pros:
- Works with varying paths lengths, allowing for flexibility 
- Can capture long term dependencies 
- Very tolerant to incomplete input data

Cons:
- Requries alot of data to be trained efficiently
- Resource intensive and long training time


CONCLUSION:
- All of the methods are effective at predicting user movements
- Markov Chains require low data to be efficient, k-nearest neighbour requires a medium amount, RNNs require a large amount
- Markov Chains have low storage and process time, RNNs and k-NN has a larger requirement (k-Nearest-Neighbours could be re-written in java or optimised in python eg: store data as a dictionary so duplicate paths are not recorded)
- For best results, a combination of all 3 should be used, especially if run-time isnt an issue and data is low
- Else, the model could change its method depending on the quantity of data 
- Organic data is required to train/test the models (random data will be inefficient)
